In [1]:
import pandas as pd
from datetime import datetime
import os,re
import json
import statistics
from collections import Counter
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
from sklearn.metrics import roc_auc_score,accuracy_score
import numpy as np
from tqdm import tqdm
from scipy import stats
from sklearn.model_selection import train_test_split

In [127]:

train=pd.read_csv('train.csv')
test=pd.read_csv('test_QyjYwdj.csv')
campaign=pd.read_csv('campaign_data.csv')
ci_mapping=pd.read_csv('coupon_item_mapping.csv')
cust_demo=pd.read_csv('customer_demographics.csv')
ct_data=pd.read_csv('customer_transaction_data.csv')
item_data=pd.read_csv('item_data.csv')

1. Merge train data with campaign data on campaign_id
2. Merge 1 with customer demographic data on customer_id
3. Merge coupon-item data with item data on item_id
4. Extract aggregated features one for each coupon_id from 3
5. Merge 2 and 4 on coupon_id
6. Merge transaction data with item data and customer demographics data on item_id and customer_id respectively
7. Extract aggregated features one for each customer_id from 6. You get a dataset with unique customer_ids.
8. Merge 5 and 7. This is the final step

In [128]:
ct_data.head()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0
1,2012-01-02,1501,54253,1,53.43,-13.89,0.0
2,2012-01-02,1501,31962,1,106.50,-14.25,0.0
3,2012-01-02,1501,33647,1,67.32,0.00,0.0
4,2012-01-02,1501,48199,1,71.24,-28.14,0.0


### As analysed in AmEx_1 only the features which are important are 
#### * quantity , selling_price , other_discount , coupon_discount,	coupon_campaign_days, campaign_type

* So let's try to join the 3 tables which contains these features are train, campaign, cutomer_transaction

In [153]:
newtrain=train.merge(campaign,on='campaign_id',how='left')
newtrain.head()

,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type,start_date,end_date
0,1,13,27,1053,0,X,19/05/13,05/07/13
1,2,13,116,48,0,X,19/05/13,05/07/13
2,6,9,635,205,0,Y,11/03/13,12/04/13
3,7,13,644,1050,0,X,19/05/13,05/07/13
4,9,8,1017,1489,0,X,16/02/13,05/04/13


In [130]:
newtest=test.merge(campaign,on='campaign_id',how='left')
newtest.head()

,id,campaign_id,coupon_id,customer_id,campaign_type,start_date,end_date
0,3,22,869,967,X,16/09/13,18/10/13
1,4,20,389,1566,Y,07/09/13,16/11/13
2,5,22,981,510,X,16/09/13,18/10/13
3,8,25,1069,361,Y,21/10/13,22/11/13
4,10,17,498,811,Y,29/07/13,30/08/13


In [131]:
ct_data1=ct_data.merge(item_data,on="item_id",how="left")

In [132]:
ct_data_grouped=ct_data1.groupby('customer_id')
ct_data_groupby=ct_data_grouped.agg({'item_id': lambda x:Counter(x).most_common()[0][0],
                                                    'quantity':'sum',
                                                    'selling_price':'sum',
                                                    'other_discount':'sum',
                                                    'coupon_discount':'sum',
                                                    'brand':lambda x:Counter(x).most_common()[0][0],
                                                    'brand_type':lambda x:Counter(x).most_common()[0][0],
                                                    'category':lambda x:Counter(x).most_common()[0][0],
                                                     'date':'count'
                                                    })
ct_data_groupby['no_of_compaigns_participated']=ct_data_grouped['date'].apply(lambda x:len(x.unique()))
ct_data_groupby['no_of_items_bought']=ct_data_grouped['item_id'].apply(lambda x:len(x.unique()))
ct_data_groupby.reset_index(inplace=True)
ct_data_groupby.head()

,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,brand,brand_type,category,date,no_of_compaigns_participated,no_of_items_bought
0,1,8307,1227,98513.93,-17030.40,-2116.83,56,Established,Grocery,1048,56,463
1,2,53517,474,43100.03,-7051.95,-249.34,56,Established,Grocery,419,37,352
2,3,32573,8163,73050.27,-16013.53,-2179.54,56,Established,Grocery,705,41,406
3,4,47968,280,33973.22,-2927.19,-89.05,56,Established,Grocery,220,26,125
4,5,35304,93353,103615.10,-10817.07,-90.83,56,Established,Grocery,792,163,490


In [147]:
cust_demo.loc[(cust_demo['no_of_children'].isnull()) & (cust_demo['family_size'] == '1')][['no_of_children','family_size']]

#there are 248 columns which can be filled as 0
cust_demo.loc[(cust_demo['no_of_children'].isnull()) & (cust_demo['family_size'] == '1'),'no_of_children']='0'



# follow the same process for marital_status as well.


cust_demo.loc[(cust_demo['family_size'] == '1') & (cust_demo['marital_status'] == "Married")][['marital_status','family_size']]

#So there are no customer is classified as married if the family size is single, so we can classify all the customers whose
# family size is 1 to marital status as Single
cust_demo.loc[(cust_demo['marital_status'].isnull()) & (cust_demo['family_size'] == '1'),'marital_status']='Single'
cust_demo.loc[(cust_demo['marital_status'].isnull()) & (cust_demo['family_size'] == '1')][['marital_status','family_size']]

#Now let's check how many null values in customer dataset
for col in cust_demo.columns:
    print("{} has {} null values".format(col,cust_demo[col].isnull().sum()))

# There are still few null values in no_of_children,marital_status let's try to fill these values
# Let's see the other fields when the marital_status is null
for col in ['age_range','family_size']:
    print(col,"\n===========")
    print(cust_demo.loc[cust_demo['marital_status'].isnull()][col].value_counts())

# So will take the mode of the combination of age_range&family_size  fields as they seem to be related

newcust_df=cust_demo.groupby(['age_range','family_size']).agg({'marital_status':lambda x: x.mode()[0] if len(x.mode()) == 2 else x.mode()}).reset_index()


for i,row in newcust_df.iterrows():
    cust_demo.loc[(cust_demo['marital_status'].isnull())&(cust_demo['age_range'] == row['age_range'])&(cust_demo['family_size']==row['family_size']),'marital_status']=row['marital_status']
    
cust_demo.head()

#If the person is married and family_size is 2 then the no_of_children should be 0
# So write a condition to filter family_size=2 & marital_status=married
cust_demo.loc[(cust_demo['marital_status']=="Married")&(cust_demo['family_size'] == '2'),'no_of_children']='0'
cust_demo['no_of_children'].isnull().sum()

#so there are still 26 records which have null values, let's see what are those

cust_demo[cust_demo['no_of_children'].isnull()]


#I am assuming that if the person's age range is greater than 26-35 
#and the marrital_status is Single he would probably married and divorced, 
# so i am assigning the no_of_children as 1 since the family size is 2 for these
cust_demo.loc[((cust_demo['age_range']).isin(['18-25','26-35'])) & (cust_demo['no_of_children'].isnull()),'no_of_children']='0'
cust_demo.loc[((cust_demo['age_range']).isin(['36-45','46-55','56-70','70+'])) & (cust_demo['no_of_children'].isnull()),'no_of_children']='1'

for col in cust_demo.columns:
    print('{} has {} null values'.format(col,cust_demo[col].isnull().sum()))

customer_id has 0 null values
age_range has 0 null values
marital_status has 146 null values
rented has 0 null values
family_size has 0 null values
no_of_children has 290 null values
income_bracket has 0 null values
age_range 
46-55    54
26-35    37
36-45    29
70+      11
18-25    10
56-70     5
Name: age_range, dtype: int64
family_size 
2     97
3     30
4     13
5+     6
Name: family_size, dtype: int64
customer_id has 0 null values
age_range has 0 null values
marital_status has 0 null values
rented has 0 null values
family_size has 0 null values
no_of_children has 0 null values
income_bracket has 0 null values


In [154]:
newtrain=newtrain.merge(cust_demo,on='customer_id',how='left')
newtrain

,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type,start_date,end_date,age_range,marital_status,rented,family_size,no_of_children,income_bracket
0,1,13,27,1053,0,X,19/05/13,05/07/13,46-55,Single,0.0,1,0,5.0
1,2,13,116,48,0,X,19/05/13,05/07/13,36-45,Married,0.0,2,0,3.0
2,6,9,635,205,0,Y,11/03/13,12/04/13,46-55,Married,0.0,2,0,7.0
3,7,13,644,1050,0,X,19/05/13,05/07/13,NaN,NaN,NaN,NaN,NaN,NaN
4,9,8,1017,1489,0,X,16/02/13,05/04/13,46-55,Married,0.0,2,0,3.0
5,11,11,795,793,0,Y,22/04/13,07/06/13,46-55,Married,0.0,2,0,5.0
6,14,9,444,590,0,Y,11/03/13,12/04/13,36-45,Married,0.0,2,0,1.0
7,15,29,538,368,0,Y,08/10/12,30/11/12,36-45,Single,0.0,1,0,5.0
8,17,30,857,523,0,X,19/11/12,04/01/13,NaN,NaN,NaN,NaN,NaN,NaN
9,19,2,559,679,0,Y,17/12/12,18/01/13,36-45,Single,1.0,2,1,4.0


In [162]:
newtrain['marital_status'].fillna(newtrain['marital_status'].mode()[0],inplace=True)
newtrain['age_range'].fillna(newtrain['age_range'].mode()[0],inplace=True)
newtrain['rented'].fillna(newtrain['rented'].mode()[0],inplace=True)
newtrain['family_size'].fillna(newtrain['family_size'].mode()[0],inplace=True)
newtrain['no_of_children'].fillna(newtrain['no_of_children'].mode()[0],inplace=True)
newtrain['income_bracket'].fillna(newtrain['income_bracket'].mode()[0],inplace=True)


In [164]:
newtest=newtest.merge(cust_demo,on='customer_id',how='left')
newtrain

newtest['marital_status'].fillna(newtest['marital_status'].mode()[0],inplace=True)
newtest['age_range'].fillna(newtest['age_range'].mode()[0],inplace=True)
newtest['rented'].fillna(newtest['rented'].mode()[0],inplace=True)
newtest['family_size'].fillna(newtest['family_size'].mode()[0],inplace=True)
newtest['no_of_children'].fillna(newtest['no_of_children'].mode()[0],inplace=True)
newtest['income_bracket'].fillna(newtest['income_bracket'].mode()[0],inplace=True)


In [163]:
newtrain2=newtrain.merge(ct_data_groupby,on='customer_id',how="left")
newtrain2.head()

,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type,start_date,end_date,age_range,marital_status,...,quantity,selling_price,other_discount,coupon_discount,brand,brand_type,category,date,no_of_compaigns_participated,no_of_items_bought
0,1,13,27,1053,0,X,19/05/13,05/07/13,46-55,Single,...,105551,57120.75,-10282.37,-89.05,56,Established,Grocery,310,80,208
1,2,13,116,48,0,X,19/05/13,05/07/13,36-45,Married,...,12143,90185.10,-10664.18,-1237.79,56,Established,Grocery,385,64,244
2,6,9,635,205,0,Y,11/03/13,12/04/13,46-55,Married,...,1351,117461.66,-17261.79,-2145.72,56,Established,Grocery,970,83,533
3,7,13,644,1050,0,X,19/05/13,05/07/13,46-55,Married,...,306,23291.42,-3947.37,-178.10,56,Established,Grocery,237,64,216
4,9,8,1017,1489,0,X,16/02/13,05/04/13,46-55,Married,...,139063,67797.49,-11534.90,-265.01,56,Established,Grocery,562,97,327


In [165]:
newtest2=newtest.merge(ct_data_groupby,on='customer_id',how="left")
newtest2.head()

,id,campaign_id,coupon_id,customer_id,campaign_type,start_date,end_date,age_range,marital_status,rented,...,quantity,selling_price,other_discount,coupon_discount,brand,brand_type,category,date,no_of_compaigns_participated,no_of_items_bought
0,3,22,869,967,X,16/09/13,18/10/13,36-45,Single,0.0,...,16644,156733.77,-16292.99,-2111.86,56,Established,Grocery,1362,67,658
1,4,20,389,1566,Y,07/09/13,16/11/13,26-35,Married,0.0,...,282396,297070.87,-32760.94,-1489.97,56,Established,Grocery,2661,89,1214
2,5,22,981,510,X,16/09/13,18/10/13,26-35,Married,0.0,...,181500,161811.27,-39450.39,-2384.75,56,Established,Grocery,1642,141,1019
3,8,25,1069,361,Y,21/10/13,22/11/13,18-25,Single,0.0,...,22196,49149.16,-7705.95,0.00,56,Established,Grocery,573,140,323
4,10,17,498,811,Y,29/07/13,30/08/13,46-55,Married,0.0,...,1729,117044.09,-19214.01,-89.04,56,Established,Grocery,1319,123,720


In [166]:
newtrain2['coupon_campaign_days']=pd.to_datetime(newtrain2['end_date'],dayfirst=True)-pd.to_datetime(newtrain2['start_date'],dayfirst=True)

newtrain2['coupon_campaign_days']=newtrain2['coupon_campaign_days'].apply(lambda x: x.days)


In [167]:
newtest2['coupon_campaign_days']=pd.to_datetime(newtest2['end_date'],dayfirst=True)-pd.to_datetime(newtest2['start_date'],dayfirst=True)

newtest2['coupon_campaign_days']=newtest2['coupon_campaign_days'].apply(lambda x: x.days)


In [173]:
newtrain3=newtrain2.drop(['id', 'campaign_id', 'coupon_id', 'customer_id','start_date','end_date','item_id'],axis=1)
newtest3=newtest2.drop(['id', 'campaign_id', 'coupon_id', 'customer_id','start_date','end_date','item_id'],axis=1)

In [174]:
get_dum_cols=['campaign_type','brand','brand_type','category','age_range','income_bracket','no_of_children','family_size','marital_status']
newtrain3=pd.get_dummies(newtrain3,columns=get_dum_cols,drop_first=True)
newtest3=pd.get_dummies(newtest3,columns=get_dum_cols,drop_first=True)

In [175]:
newtrain3.columns

Index(['redemption_status', 'rented', 'quantity', 'selling_price',
       'other_discount', 'coupon_discount', 'date',
       'no_of_compaigns_participated', 'no_of_items_bought',
       'coupon_campaign_days', 'campaign_type_Y', 'brand_63', 'brand_89',
       'brand_133', 'brand_158', 'brand_278', 'brand_303', 'brand_369',
       'brand_482', 'brand_487', 'brand_524', 'brand_659', 'brand_681',
       'brand_779', 'brand_838', 'brand_892', 'brand_923', 'brand_946',
       'brand_967', 'brand_1075', 'brand_1088', 'brand_1101', 'brand_1124',
       'brand_1337', 'brand_1447', 'brand_1562', 'brand_1564', 'brand_1996',
       'brand_2025', 'brand_2084', 'brand_2370', 'brand_2379', 'brand_4991',
       'brand_type_Local', 'category_Grocery', 'category_Packaged Meat',
       'category_Pharmaceutical', 'age_range_26-35', 'age_range_36-45',
       'age_range_46-55', 'age_range_56-70', 'age_range_70+',
       'income_bracket_2.0', 'income_bracket_3.0', 'income_bracket_4.0',
       'income_brack

In [177]:
X=newtrain3.drop('redemption_status',axis=1)
y=newtrain3['redemption_status']
param_lgb = { 'boosting_type': 'gbdt', 'objective': 'binary', 'metric':'auc',
             'bagging_freq':1, 'subsample':1, 'feature_fraction': 0.7,
              'num_leaves': 8, 'learning_rate': 0.05, 'lambda_l1':5,'max_bin':255}
# form LightGBM datasets
dtrain_lgb = lgb.Dataset(X, label=y)
# LightGBM, cross-validation
cv_result_lgb = lgb.cv(param_lgb, 
                       dtrain_lgb, 
                       num_boost_round=1000, 
                       nfold=20, 
                       stratified=True, 
                       early_stopping_rounds=15, 
                       verbose_eval=100, 
                       show_stdv=True)

[100]	cv_agg's auc: 0.878759 + 0.0242008
[200]	cv_agg's auc: 0.884601 + 0.0230839
[300]	cv_agg's auc: 0.887062 + 0.0224103
[400]	cv_agg's auc: 0.889042 + 0.0214832
[500]	cv_agg's auc: 0.889923 + 0.0212531


array([438, 333, 391, 563, 455, 486, 415, 328,  43,  13,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   1,   3,   0,   0,   0,   0])

In [178]:

model_lgb = lgb.train(param_lgb, dtrain_lgb, num_boost_round=len(cv_result_lgb['auc-mean']))
y_pred=model_lgb.predict(newtest3)

In [179]:
model_lgb.feature_importance()

array([  0, 435, 338, 437, 573, 443, 475, 419, 319,  56,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   1,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   1,   0,   0,   0,  11,   1,  13,   0,   3,   1,
         0,   7,  13,   1,   0,   0,   0,   2,   0,   0,   4,   0,  11,
        11,   2,   0,   0,  35])

In [182]:
X.columns

Index(['rented', 'quantity', 'selling_price', 'other_discount',
       'coupon_discount', 'date', 'no_of_compaigns_participated',
       'no_of_items_bought', 'coupon_campaign_days', 'campaign_type_Y',
       'brand_63', 'brand_89', 'brand_133', 'brand_158', 'brand_278',
       'brand_303', 'brand_369', 'brand_482', 'brand_487', 'brand_524',
       'brand_659', 'brand_681', 'brand_779', 'brand_838', 'brand_892',
       'brand_923', 'brand_946', 'brand_967', 'brand_1075', 'brand_1088',
       'brand_1101', 'brand_1124', 'brand_1337', 'brand_1447', 'brand_1562',
       'brand_1564', 'brand_1996', 'brand_2025', 'brand_2084', 'brand_2370',
       'brand_2379', 'brand_4991', 'brand_type_Local', 'category_Grocery',
       'category_Packaged Meat', 'category_Pharmaceutical', 'age_range_26-35',
       'age_range_36-45', 'age_range_46-55', 'age_range_56-70',
       'age_range_70+', 'income_bracket_2.0', 'income_bracket_3.0',
       'income_bracket_4.0', 'income_bracket_5.0', 'income_bracket_6.0

In [123]:
sample_submission=pd.read_csv('sample_submission2.csv')
sample_submission.head()

,id,redemption_status
0,3,0.036027
1,4,0.008227
2,5,0.050259
3,8,0.000193
4,10,0.000203


In [180]:
sample_submission['redemption_status']=y_pred
sample_submission.head()

,id,redemption_status
0,3,0.027868
1,4,0.007321
2,5,0.051074
3,8,0.000233
4,10,0.000225


In [181]:
sample_submission.to_csv('sample_submission5.csv',index=False)